# ImgLib2 

Example 07: Interpolation

## Introduction

Interpolation is a basic operation required in many image processing tasks. In the terminology of ImgLib2 it means to convert a [**RandomAccessible**](http://javadoc.imagej.net/ImgLib2/net/imglib2/RandomAccessible.html) into a [**RealRandomAccessible**](http://javadoc.imagej.net/ImgLib2/net/imglib2/RealRandomAccessible.html) which is able to create a [**RealRandomAccess**](http://javadoc.imagej.net/ImgLib2/net/imglib2/RealRandomAccess.html). It can be positioned at real coordinates instead of only integer coordinates and return a value for each real location. 

Currently, three interpolation schemes are available for ImgLib2:
  * Nearest neighbor interpolation (also available for any kind of data that can return a nearest neighbor like sparse datasets)
  * Linear interpolation
  * Lanczos interpolation

In the example, we magnify a given real interval in the [RealRandomAccessible](http://javadoc.imagej.net/ImgLib2/net/imglib2/RealRandomAccessible.html) which is based on the interpolation on an [Img](http://javadoc.imagej.net/ImgLib2/net/imglib2/img/Img.html) and compare the results of all three interpolation methods.

## Bootstrap ImageJ

In [ ]:
// Behind a firewall? Configure your proxy settings here.
//systemProperties = System.getProperties()
//systemProperties.setProperty("http.proxyHost","myproxy.domain")
//systemProperties.setProperty("http.proxyPort","8080")

//////////////////////////////////////////////////////////////
// Load ImageJ. This may take some minutes the first time   //
// while ImageJ is installed into ~/.groovy/grapes locally. //
//////////////////////////////////////////////////////////////
@GrabResolver(name='imagej', root='http://maven.imagej.net/content/groups/public/')
@Grab('net.imagej:imagej:2.0.0-rc-58')
import net.imagej.ImageJ
ij = new ImageJ()
println("ImageJ is ready to go.")

Below is the utility method for a given interval magnification.

In [ ]:
import net.imglib2.type.Type
import net.imglib2.RealRandomAccessible
import net.imglib2.RealInterval
import net.imglib2.img.ImgFactory
import net.imglib2.img.Img

mag = new Object() {
  /**
   * Compute a magnified version of a given real interval
   *
   * @param <T> value type
   * @param source - the input data
   * @param interval - the real interval on the source that should be
   * magnified
   * @param factory - the image factory for the output image
   * @param magnification - the ratio of magnification
   * @return - an Img that contains the magnified image content
   */
  public <T extends Type<T>> Img<T> magnify(
    RealRandomAccessible<T> source,
    RealInterval interval,
    ImgFactory<T> factory,
    double magnification) {
    
    numDimensions = interval.numDimensions()
    
    // compute the number of pixels of the output and the size of the real interval
    pixelSize = new long[numDimensions]
    intervalSize = new double[numDimensions]
    
    for (int d = 0; d < numDimensions; ++d) {
      intervalSize[d] = interval.realMax(d) - interval.realMin(d)
      pixelSize[d] = Math.round(intervalSize[d] * magnification) + 1
    }
    
    // create the output image
    output = factory.create(pixelSize, source.realRandomAccess().get())
    
    // cursor to iterate over all pixels
    cursor = output.localizingCursor()
    
    // create a RealRandomAccess on the source (interpolator)
    realRandomAccess = source.realRandomAccess()
    
    // the temporary array to compute the position
    tmp = new double[numDimensions]
    
    // for all pixels of the output image
    while (cursor.hasNext()) {
      cursor.fwd()
      
      // compute the appropriate location of the interpolator
      for (int d = 0; d < numDimensions; ++d) {
        tmp[d] = cursor.getDoublePosition(d) / output.realMax(d) * intervalSize[d] + interval.realMin(d)
      }
      
      // set the position
      realRandomAccess.setPosition(tmp)
      
      // set the new value
      cursor.get().set(realRandomAccess.get())
    }
    
    return output
  }
}

The below code uses three different interpolators to 10x magnify a small area.

In [ ]:
import io.scif.img.ImgOpener
import net.imglib2.FinalRealInterval
import net.imglib2.type.numeric.real.FloatType
import net.imglib2.interpolation.randomaccess.LanczosInterpolatorFactory
import net.imglib2.interpolation.randomaccess.NLinearInterpolatorFactory
import net.imglib2.interpolation.randomaccess.NearestNeighborInterpolatorFactory
import net.imglib2.img.array.ArrayImgFactory
import net.imglib2.view.Views

enum InterpolationType {
  NEAREST_NEIGHBOR_INTERPOLATION,
  NLINEAR_INTERPOLATION,
  LANCZOS_INTERPOLATION
}

// open with ImgOpener as a FloatType
image = new ImgOpener().openImgs("http://samples.fiji.sc/tutorials/DrosophilaWing.tif", new FloatType()).get(0)

// define the area in the interpolated image
double[]min = [105.12, 40.43]
double[]max = [129.56, 74.933]

interval = new FinalRealInterval(min, max)

// change this variable to switch for different kind of interpolation scheme display
interpolant_type = InterpolationType.NEAREST_NEIGHBOR_INTERPOLATION
//interpolant_type = InterpolationType.NLINEAR_INTERPOLATION
//interpolant_type = InterpolationType.LANCZOS_INTERPOLATION

// create a RandomAccessible using the factory and views method
// it is important to extend the image first, the interpolation scheme might
// grep pixels outside of the boundaries even when locations inside are queried
// as they integrate pixel information in a local neighborhood - the size of
// this neighborhood depends on which interpolator is used
switch (interpolant_type) {
  case InterpolationType.NEAREST_NEIGHBOR_INTERPOLATION:
    // create an InterpolatorFactory RealRandomAccessible using nearst neighbor interpolation
    nn_inter_factory = new NearestNeighborInterpolatorFactory<FloatType>()
    nn_interpolant = Views.interpolate(Views.extendMirrorSingle(image), nn_inter_factory)
    img = mag.magnify(nn_interpolant, interval, new ArrayImgFactory<FloatType>(), 10)
    break
  case InterpolationType.NLINEAR_INTERPOLATION:
    // create an InterpolatorFactory RealRandomAccessible using linear interpolation
    linear_inter_factory = new NLinearInterpolatorFactory<FloatType>()
    nlinear_interpolant = Views.interpolate(Views.extendMirrorSingle(image), linear_inter_factory)
    img = mag.magnify(nlinear_interpolant, interval, new ArrayImgFactory<FloatType>(), 10)
    break
  case InterpolationType.LANCZOS_INTERPOLATION:
    // create an InterpolatorFactory RealRandomAccessible using lanczos interpolation
    lanczos_inter_factory = new LanczosInterpolatorFactory<FloatType>()
    lanczos_interpolant = Views.interpolate(Views.extendMirrorSingle(image), lanczos_inter_factory)
    img = mag.magnify(lanczos_interpolant, interval, new ArrayImgFactory<FloatType>(), 10)
    break
  default:
    nn_inter_factory = new NearestNeighborInterpolatorFactory<FloatType>()
    nn_interpolant = Views.interpolate(Views.extendMirrorSingle(image), nn_inter_factory)
    img = mag.magnify(nn_interpolant, interval, new ArrayImgFactory<FloatType>(), 10)
}

ij.notebook().display(img)

The nearest neighbor interpolation is the fastest to compute and is the most versatile as it requires no computation, but just a lookout. The result is, however, very pixelated. The linear interpolation produces reasonable results and computes quite fast. The Lanczos interpolation shows the most visually pleasing results, but also introduces slight artifacts in the background.